# Advanced Querying Mongo

Importing libraries and setting up connection

In [3]:
from pymongo import MongoClient
client = MongoClient("mongodb://localhost/companies")

db = client.get_database()

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [100]:
# Your Code
babelgum = db["companies"].find({"name": "Babelgum"}, {"name": 1, "_id": 0})
list(babelgum)

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [104]:
# Your Code
employees = db["companies"].find({"number_of_employees": { "$gte": 5000 } }, {"name":1, "_id": 0}).sort([("number_of_employees", -1)]).limit(20)
list(employees[1:])

[{'name': 'IBM'},
 {'name': 'Toyota'},
 {'name': 'PayPal'},
 {'name': 'Nippon Telegraph and Telephone Corporation'},
 {'name': 'Samsung Electronics'},
 {'name': 'Accenture'},
 {'name': 'Tata Consultancy Services'},
 {'name': 'Flextronics International'},
 {'name': 'Safeway'},
 {'name': 'Sony'},
 {'name': 'LG'},
 {'name': 'Ford'},
 {'name': 'Boeing'},
 {'name': 'Digital Equipment Corporation'},
 {'name': 'Nokia'},
 {'name': 'MItsubishi Electric'},
 {'name': 'MItsubishi Electric'},
 {'name': 'Comcast'},
 {'name': 'Bertelsmann'},
 {'name': 'MetaCarta'},
 {'name': 'General Dynamics'},
 {'name': 'Microsoft'},
 {'name': 'CSC'},
 {'name': 'CSC'},
 {'name': 'Intel'},
 {'name': 'Cox Enterprises'},
 {'name': 'Apple'},
 {'name': 'Dell'},
 {'name': '3M'},
 {'name': 'Alcatel-Lucent'},
 {'name': 'ExxonMobil'},
 {'name': 'Affiliated Computer Services'},
 {'name': 'CGI Group'},
 {'name': 'Cisco'},
 {'name': 'Xerox'},
 {'name': 'Motorola Solutions'},
 {'name': 'Thomson Reuters'},
 {'name': 'Thomson Reu

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [99]:
# Your Code
years = [e for e in range(2000, 2006)]
founded = db["companies"].find({"founded_year" : {"$in":years}}, {"name":1, "founded_year": 1, "_id": 0})
list(founded)

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Omnidrive', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Gizmoz', 'founded_year': 2003},
 {'name': 'Helio', 'founded_year': 2005},
 {'name': 'StumbleUpon', 'founded_year': 2002},
 {'name': 'Plaxo', 'founded_year': 2002},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Technorati', 'founded_year': 2002},
 {'name': 'Veoh', 'founded_year': 2004},
 {'name': 'SmugMug', 'founded_year': 2002},
 {'name': 'Wesabe', 'founded_year': 2005},
 {'name': 'Jangl SMS', 'founded_year': 2005},
 {'name': 'Jingle Networks', 'founded_year': 2005},
 {'name': 'Meetup', 'founded_year': 2002},
 {'name': 'AddThis', 'founded_year': 2004},
 {'name': 'LifeLock', 'founded_year': 2005},
 {'name': 'Jajah', 'founded_year': 2005},
 {'name': 'Pando Networks', 'founded_year': 2004},
 {'name': 'Ikan', 'founded_year': 2003},
 {'name': 'YouTube', 'founded_year': 2005},
 {'name': 'Topix', 'foun

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [109]:
# Your Code
valuation = db["companies"].find({"$and":[{"ipo.valuation_amount":{"$gt": 100000000}},{"founded_year":{"$lt": 2010}}]}, {"name":1, "ipo": 1, "_id": 0})
list(valuation)



[{'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}},
 {'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'name': 'Yelp',
  'ipo': {'valuation_amount': 1300000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 3,
   'pub_day': 2,
   'stock_symbol': 'NYSE:YELP'}},
 {'name': 'LinkedIn',
  'ipo': {'valuation_amount': 9310000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2011,
   'pub_month': 7,
   'pub_day': 20,
   'stock_symbol': 'NYSE:LNKD'}},
 {'name': 'Brightcove',
  'ipo': {'valuation_amount': 290000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 2,
   'pub_day': 17,
   'stock_symbol': 'NASDAQ:BCOV'}},
 {'name': 'Amazon',
  'ipo': {'valu

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [111]:
# Your Code
lessEmp = db["companies"].find({"$and":[{"number_of_employees":{"$lt": 1000}},{"founded_year":{"$lt": 2005}}]}, {"name":1,"_id": 0}).sort([("number_of_employees", -1)]).limit(10)
list(lessEmp)

[{'name': 'Infinera Corporation'},
 {'name': 'NorthPoint Communications Group'},
 {'name': '888 Holdings'},
 {'name': 'Forrester Research'},
 {'name': 'SonicWALL'},
 {'name': 'Webmetrics'},
 {'name': 'Cornerstone OnDemand'},
 {'name': 'Yelp'},
 {'name': 'ZoomInfo'},
 {'name': 'MySpace'}]

### 6. All the companies that don't include the `partners` field.

In [134]:
# Your Code
partners = db["companies"].find({"partners": {"$exists": False}}, {"name": 1, "_id": 0})
list(partners.limit(5))

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [144]:
# Your Code
category = db["companies"].find({"category_code": {"$type": "null"}}, {"name": 1, "_id": 0})
list(category) 

[{'name': 'Collective'},
 {'name': 'Snimmer'},
 {'name': 'KoolIM'},
 {'name': 'Level9 Media'},
 {'name': 'VidKing'},
 {'name': 'Drigg'},
 {'name': 'SpaceTime'},
 {'name': 'MMDAYS'},
 {'name': 'Touch Clarity'},
 {'name': 'Inside Group'},
 {'name': 'Repeater Store'},
 {'name': 'Tapesh'},
 {'name': 'iPersians'},
 {'name': 'NewPersia'},
 {'name': 'Pyra Labs'},
 {'name': 'Feedmap'},
 {'name': 'NuvoMedia'},
 {'name': 'Intwine'},
 {'name': 'The Weinstein Company'},
 {'name': 'ExecuNet'},
 {'name': 'TripHomes'},
 {'name': 'A1Vacations'},
 {'name': 'Picturehouse'},
 {'name': 'Covalent Technologies'},
 {'name': 'Epic Trip'},
 {'name': 'Telepost'},
 {'name': 'Fidelity'},
 {'name': 'Quirk'},
 {'name': 'Skyrove'},
 {'name': 'Leechblock'},
 {'name': 'My Success Gateway'},
 {'name': 'The Easynet Group'},
 {'name': 'NetNames'},
 {'name': 'RealNames'},
 {'name': 'Sniperoo'},
 {'name': 'JD Project'},
 {'name': 'ABC News'},
 {'name': 'Teoma Technologies'},
 {'name': 'cometro'},
 {'name': 'U-Cast'},
 {'na

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [ ]:
# Your Code
emplo = db

### 9. Order all the companies by their IPO price in a descending order.

In [ ]:
# Your Code

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [ ]:
# Your Code

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [ ]:
# Your Code

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [ ]:
# Your Code

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [ ]:
# Your Code

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [ ]:
# Your Code

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [ ]:
# Your Code

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [ ]:
# Your Code

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [ ]:
# Your Code

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [ ]:
# Your Code

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code